In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
go_headers={
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 
    'Accept-Encoding' : 'gzip, deflate, br', 
    'Accept-Language' : 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7', 
    'Cache-Control' : 'max-age=0', 
    'Connection' : 'keep-alive', 
    'Cookie' : 'gdpr=0; _ym_uid=1650040415881014180; spravka=dD0xNjUwMDQwNDE3O2k9OTUuMzEuMTY0LjIwNDtEPTZCN0IyMzg0QjgxMDhFRDUwQjE2ODJDODM1MEM5MDVCNUNBNzE0RDQxRTQ2MzVBMUU0MDZFN0EzRjUzNkFCMTY1RDdBOTk2Qzt1PTE2NTAwNDA0MTc3MDkwMTAxMjk7aD02ZDA1Zjk0YmM2ZTBlNjYzMGI4YWQxMzRjY2U2YTYyMQ==; _csrf_token=05526f7dc250b81c6ad8a6fe234afedab0f64fd54726108f; suid=beb23d545bdf34b6846c46043cb502d3.27f1dd7bf67e250b0d265b5f6b7e3562; from=direct; yuidlt=1; yandexuid=4560098021572630586; my=YwA%3D; ys=udn.cDp5YWdvbmNoYXJvdmExNTEyMjU%3D%23c_chck.3729760260; autoru_sid=a%3Ag62599e62257ibr3vhsohmp9phq27ook.b87b08f76d8c3e8622bf276898b0f84b%7C1650040418338.604800.pRLucir6kCSl0INJLfO1KA.FZgwNkU425AyH1gkx7Ri7MN6isl4r4YqtH4R-YL579g; autoruuid=g62599e62257ibr3vhsohmp9phq27ook.b87b08f76d8c3e8622bf276898b0f84b; crookie=AMxwNNvnHFBycGTqFyhFX+CbR2dC7trC/w99WUyAeR6OWq2kd3Hzhl+WVpdoaAaJWr4Zbork7Lvll4/6k4+HCIGSLFc=; cmtchd=MTY1MDA0MDQyMTk4Mw==; _ym_isad=1; los=1; bltsr=1; gids=1%2C213; Session_id=3:1650042555.5.3.1631103548565:SmmzWQ:27.1.2:1|563763314.13423332.2.2:13423332|1327063823.118819.2.2:118819|1015261777.1995054.2.2:1995054|844574756.3271684.2.2:3271684|1118691669.13466947.2.2:13466947|61:3753.813876.HKAAYDRSA3QDBRE6n2EZtoQ4gVI; yandex_login=yagoncharova151225; i=WWcVfXvVydauEmguFL5w3EHm4kCn53EOQ1QfIs9zBVdQel0tSu7pUMc7RbO43Iwo8JDzKXp7Z7MwM1Ah/Pm6MNzkZhQ=; mda2_beacon=1650042555323; sso_status=sso.passport.yandex.ru:blocked; from_lifetime=1650052923558; _yasc=7tn5xs2WaMIEdJGkp4zRFfHmura+xeIyGfLTYNBHUc+NR1qM; _ym_d=1650052957', 
    'Host' : 'auto.ru', 
    'sec-ch-ua' : '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"', 
    'sec-ch-ua-mobile' : '?0', 
    'sec-ch-ua-platform' : '"Windows"', 
    'Sec-Fetch-Dest' : 'document', 
    'Sec-Fetch-Mode' : 'navigate', 
    'Sec-Fetch-Site' : 'none', 
    'Sec-Fetch-User' : '?1', 
    'Upgrade-Insecure-Requests' : '1', 
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
}

In [71]:
def pars_page(i_numberPage):
    
    df = pd.DataFrame()
    with requests.Session() as lo_s:

        lo_r = lo_s.get("https://auto.ru/cars/used/?year_from=2021&page="+str(i_numberPage), headers = go_headers)
        if (lo_r.status_code == 200):
            #print(lo_r.encoding)
            lo_r.encoding = 'utf-8'
            #print(lo_r.encoding)
            soup = BeautifulSoup(lo_r.text, "html.parser")
            allcar = soup.findAll('div', class_='ListingItem__main')
        
            for data in allcar:
                try:
                    dic = {}
                    dic["Авто"] = data.find('a', class_='Link ListingItemTitle__link').text
                    lst_dv = data.findAll('div', class_='ListingItemTechSummaryDesktop__cell')
                    dic["Двигатель"] = lst_dv[0].text
                    dic["КПП"] = lst_dv[1].text
                    dic["Кузов"] = lst_dv[2].text
                    dic["Привод"] = lst_dv[3].text
                    dic["Цвет"] = lst_dv[4].text
                    dic["Цена"] = data.find('a', class_='Link ListingItemPrice__link').text
                    dic["Год выпуска"] = data.find('div', class_='ListingItem__year').text
                    dic["Пробег"] = data.find('div', class_='ListingItem__kmAge').text
                    #print(dic)
                    df = df.append(dic, ignore_index=True)
                except Exception:
                    print('Это что ещё такое?')
            df.head()

        else:
            print("Error")
    return df

In [72]:
df_full = pd.DataFrame()
for i in range(2, 10):
    print(i)
    a = pars_page(i)
    df_full = df_full.append(a)
df_full.to_excel('много страниц.xlsx')

2
3
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
4
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
5
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
6
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
Это что ещё такое?
7
Это что ещё такое?
8
Это что ещё такое?
9
Это что ещё такое?
